# BASICS 03 - Training Classifier with MLR (or Histogram Comparison)

# TODO:
- make a dataset builder from the output of HOTS
- implement a classifier with MLR in torch
- implement a classifier with HC in torch
- implement an accuracy measure

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

from network import network
from tools import HOTS_Dataset, get_loader, fit_MLR, predict_MLR
import os, tonic

/home/INT/grimaldi.a/Documents/projets/HOTS/newhots/HOTS


In [5]:
train = True
original_dataset = tonic.datasets.POKERDVS(save_to='../../Data', train=train, transform = tonic.transforms.NumpyAsType(int))

In [6]:
#for poker DVS
sensor_size = original_dataset.sensor_size
name = 'homhots'
tau = (0.7,1.4,2.8)
homeo = (2,1)
timestr = '2021-12-17_poker'
tau_cla = 5e3

In [7]:
hots = network(name = name, tau = tau, homeo = homeo, timestr = timestr, camsize=(sensor_size[0], sensor_size[1]))

In [8]:
model, loss = fit_MLR(hots, tau_cla)

Number of training samples: 47
load existing model


In [9]:
likelihood, true_target = predict_MLR(hots, model, tau_cla)

In [10]:
i = 0

In [31]:
import matplotlib.pyplot as plt
import numpy as np

print(true_target[i])
plt.plot(likelihood[i][:,true_target[i]], 'green')
others = np.arange(len(original_dataset.classes))
others = np.delete(others, true_target[i])
plt.plot(likelihood[i][:,others], 'red')
i+=1

IndexError: list index out of range

In [ ]:
# il n'arrive pas à reconnaitre la classe 1 -> di (carreau)
# ça vient surement d'un problème de clustering -> règle d'homéostasie trop forte (à vérifier)